features:

log mass1_ratio, log mass2_ratio, max e, sd e, cv peri

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import corner
import math
import csv
from numpy import *

import matplotlib
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from tqdm.notebook import tqdm

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import sklearn.metrics as metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data
from IPython.display import Image
from fpdf import FPDF
from PIL import Image
import pydotplus

%matplotlib inline
import graphviz
from sklearn import tree
from xgboost import plot_tree
from sklearn.tree import DecisionTreeClassifier

In [97]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]

### the wrong way

In [103]:
sys = dict()
te = dict()


for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['pericenter'] = []
        sys[rn2][planet]['log mass1_ratio'] = []
        sys[rn2][planet]['log mass2_ratio'] = []
        sys[rn2][planet]['sd e'] = []
        sys[rn2][planet]['max e'] = []
        sys[rn2][planet]['cv pericenter'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(np.log10(data[rn2][planet][timestep]['e'])))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:    
                if (planet == lm2):
                    m1 = float(data[rn2][lm3][0]['mass'])
                    a1 = float(data[rn2][lm3][timestep]['a'])

                    m2 = float(data[rn2][lm4][0]['mass'])
                    a2 = float(data[rn2][lm4][timestep]['a'])

                elif (planet == lm3):
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm4][0]['mass'])
                    a2 = float(data[rn2][lm4][timestep]['a'])

                else:
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])
                    
                sys[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                sys[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                sys[rn2][planet]['cv pericenter'].append(np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])]))
                sys[rn2][planet]['sd e'].append(np.std(te[rn2][planet]['e']))
                sys[rn2][planet]['max e'].append(np.max(te[rn2][planet]['e']))
                
                
            except:
                pass
            
            
            
                
for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

In [128]:
feats = ['log mass1_ratio', 'log mass2_ratio', 'max e', 'sd e', 'cv pericenter']


xs = []

for f in range(len(sys[1][1]['max e'])):
    xs.append(f * 13.689)    
    
    
pbar = tqdm(total = 100)
fil = PdfPages('time stamping 1.pdf')


for s in range(1,101):    
    fig, axs = plt.subplots(5, 1, figsize=(7,17), sharex=True)
    plt.suptitle('System ' + str(s), fontweight='bold')

    for f in range(len(feats)):
        for p in range(1,11):
            col = 'white'
            
            if sys[s][p]['fate'] == 'remaining':
                col = 'green'
            else:
                col = 'lightsteelblue'
            
            try:
                axs[f].plot(xs, sys[s][p][feats[f]], color = col)
            except:
                xs2 = []
                
                for i in range(len(sys[s][p][feats[f]])):
                    xs2.append(i * 13.689)
                
                axs[f].plot(xs2, sys[s][p][feats[f]], color = col)

            
        axs[0].set_ylabel('log mass1 ratio')
        axs[1].set_ylabel('log mass2 ratio')
        axs[2].set_ylabel('max e')
        axs[3].set_ylabel('sd e')
        axs[4].set_ylabel('cv pericenter')
        

    plt.tight_layout()
    plt.subplots_adjust(top=0.96)
    
    fil.savefig()
    plt.clf()
    pbar.update()
    
fil.close()

<ipython-input-128-82390f785130>:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(5, 1, figsize=(7,17), sharex=True)


<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

...

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

### attempt number 2

In [140]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=20)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]

In [152]:
sys = dict()
te = dict()


for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(np.log10(data[rn2][planet][timestep]['e'])))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:    
                if (planet == lm2):
                    m1 = float(data[rn2][lm3][0]['mass'])
                    a1 = float(data[rn2][lm3][timestep]['a'])

                    m2 = float(data[rn2][lm4][0]['mass'])
                    a2 = float(data[rn2][lm4][timestep]['a'])

                elif (planet == lm3):
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm4][0]['mass'])
                    a2 = float(data[rn2][lm4][timestep]['a'])

                else:
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])
                    
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass
            

            
for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['sd e'] = []
        sys[rn2][planet]['max e'] = []
        sys[rn2][planet]['cv pericenter'] = []
        sys[rn2][planet]['log mass1_ratio'] = []
        sys[rn2][planet]['log mass2_ratio'] = []
        
        for i in range(0, 11):
            sys[rn2][planet]['log mass1_ratio'].append(np.average(te[rn2][planet]['log mass1_ratio'][i:i+10]))
            sys[rn2][planet]['log mass2_ratio'].append(np.average(te[rn2][planet]['log mass2_ratio'][i:i+10]))
            sys[rn2][planet]['sd e'].append(np.std(te[rn2][planet]['e'][i:i+10]))
            sys[rn2][planet]['cv pericenter'].append(np.std(te[rn2][planet]['pericenter'][i:i+10])/np.mean(te[rn2][planet]['pericenter'][i:i+10]))
            
            try:
                for em in range(i, i+10):
                    temp = []
                    temp.append(te[rn2][planet]['e'][em])

                sys[rn2][planet]['max e'].append(np.max(temp))
            except:
                pass
                
                
for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

In [156]:
feats = ['log mass1_ratio', 'log mass2_ratio', 'max e', 'sd e', 'cv pericenter']


xs = []

for f in range(len(sys[1][1]['max e'])):
    xs.append(f * 13.689)    
    
    
pbar = tqdm(total = 100)
fil = PdfPages('time stamping 2.pdf')


for s in range(1,101):    
    fig, axs = plt.subplots(5, 1, figsize=(7,17), sharex=True)
    plt.suptitle('System ' + str(s), fontweight='bold')

    for f in range(len(feats)):
        for p in range(1,11):
            col = 'white'
            
            if sys[s][p]['fate'] == 'remaining':
                col = 'green'
            else:
                col = 'lightsteelblue'
            
            try:
                axs[f].plot(xs, sys[s][p][feats[f]], color = col)
            except:
                xs2 = []
                
                for i in range(len(sys[s][p][feats[f]])):
                    xs2.append(i * 13.689)
                
                axs[f].plot(xs2, sys[s][p][feats[f]], color = col)

            
        axs[0].set_ylabel('log mass1 ratio')
        axs[1].set_ylabel('log mass2 ratio')
        axs[2].set_ylabel('max e')
        axs[3].set_ylabel('sd e')
        axs[4].set_ylabel('cv pericenter')
        

    plt.tight_layout()
    plt.subplots_adjust(top=0.96)
    
    fil.savefig()
    plt.clf()
    pbar.update()
    
fil.close()

<ipython-input-156-2e771c9fc05d>:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(5, 1, figsize=(7,17), sharex=True)


<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

...

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

<Figure size 504x1224 with 0 Axes>

## revised attempt number 2

#### with lines colored by classification

In [156]:
planets = pd.read_csv('2-3 mm.csv', skipinitialspace=True)

fates = list(set(planets['fate']))

types = {fates[i] : i for i in range(len(fates))}

classes = []

for i in range(len(planets['fate'])):
    if planets['fate'][i] == "remaining":
        classes += [0]
    else:
        classes += [1]
        
classes = np.array(classes)

In [161]:
features_train, features_test, classes_train, classes_test = train_test_split(planets, classes, test_size=0.3, random_state=1)

ru = features_test['run'].to_numpy()
p = features_test['planet'].to_numpy()
e = features_test['end time'].to_numpy()
fa = features_test['fate'].to_numpy()

ru2 = features_train['run'].to_numpy()
p2 = features_train['planet'].to_numpy()
e2 = features_train['end time'].to_numpy()
fa2 = features_train['fate'].to_numpy()

d = ['run', 'planet', 'fate', 'end time', 'initial pericenter',
       'final pericenter', 'mean pericenter', 'min pericenter',
       'max pericenter', 'sd pericenter', 'initial jacobi', 'final jacobi',
       'mean jacobi', 'min jacobi', 'max jacobi', 'sd jacobi', 'final log mass1_ratio',
       'mean log mass1_ratio', 'min log mass1_ratio', 'max log mass1_ratio',
       'sd log mass1_ratio', 'final log mass2_ratio', 
       'mean log mass2_ratio', 'min log mass2_ratio',
       'max log mass2_ratio', 'sd log mass2_ratio']


e = ['e']
mhr1 = ['mhr1']
mhr2 = ['mhr2']
a2 = ['log a2_ratio']

a1 = ['log a1_ratio']
o = ['i', 'a']

for v in mhr1:
    d.append('initial ' + str(v))
    d.append('final ' + str(v))
    d.append('mean ' + str(v))
    d.append('min ' + str(v))
#     d.append('max ' + str(v))
#     d.append('sd ' + str(v))

for v in mhr2:
    d.append('initial ' + str(v))
    d.append('final ' + str(v))
    d.append('mean ' + str(v))
    d.append('min ' + str(v))
    d.append('max ' + str(v))
#     d.append('sd ' + str(v))

for v in e:
    d.append('initial ' + str(v))
    d.append('final ' + str(v))
    d.append('mean ' + str(v))
#     d.append('min ' + str(v))
#     d.append('max ' + str(v))
#     d.append('sd ' + str(v))
    d.append('cv ' + str(v))
    
for v in a2:
    d.append('initial ' + str(v))
    d.append('final ' + str(v))
    d.append('mean ' + str(v))
    d.append('min ' + str(v))
    d.append('max ' + str(v))
    d.append('sd ' + str(v))

for v in a1:
    d.append('initial ' + str(v))
    d.append('final ' + str(v))
#     d.append('mean ' + str(v))
#     d.append('min ' + str(v))
    d.append('max ' + str(v))
#     d.append('sd ' + str(v))

for v in o:
    if v == 'i':
        d.append('initial ' + str(v))
        d.append('final ' + str(v))
        d.append('mean ' + str(v))
        d.append('min ' + str(v))
        d.append('max ' + str(v))
#         d.append('sd ' + str(v))
#         d.append('cv ' + str(v))
    else:
        d.append('cv ' + str(v))
            
      
ids_train = features_train['planet'].to_numpy()
features_train.drop(d, axis=1, inplace=True)

cols = features_train.columns
features_train = features_train.to_numpy()

ids_test = features_test['planet'].to_numpy()
features_test.drop(d, axis=1, inplace=True)
features_test = features_test.to_numpy()

/Users/neha/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [162]:
classifier = GradientBoostingClassifier(random_state=1)

classifier.fit(features_train, classes_train)

score = classifier.decision_function(features_test)
classes_predict = classifier.predict(features_test)

In [163]:
pred = cross_val_predict(classifier, features_train, classes_train, cv=10)

fate = []

for i in range(len(fa)):
    if (fa[i] == 'remaining'):
        fate.append(0)
    else:
        fate.append(1)
        
        
        
test_tup = [(ru[i], p[i]) for i in range (len(ru))]
all_dict = {}
for i in range(len(ru)):
    tt = test_tup[i]
    all_dict[tt] = {}
    all_dict[tt]['outcome'] = classes_test[i]
    all_dict[tt]['predict'] = classes_predict[i]
    all_dict[tt]['type'] = 'test'
    
    
test_tup = [(ru2[i], p2[i]) for i in range (len(ru2))]    
for i in range(len(ru2)):
    tt = test_tup[i]
    all_dict[tt] = {}
    all_dict[tt]['outcome'] = classes_train[i]
    all_dict[tt]['predict'] = pred[i]
    all_dict[tt]['type'] = 'train'

#### graphing

In [ ]:
feats = ['log mass1_ratio', 'log mass2_ratio', 'max e', 'sd e', 'cv pericenter']


pbar = tqdm(total = 100)
fil = PdfPages('time stamping revised all feats.pdf')

for rn in range(1,101):
    data = dict()
    
    for planetnum in range(1,11):
        data[planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=36561)
        
        for time in range(len(pl)):
            data[planetnum][time] = dict()
            
            try:
                data[planetnum][time]['mass'] = pl[time][1]
                data[planetnum][time]['a'] = pl[time][2]
                data[planetnum][time]['e'] = pl[time][3]
            except:
                data[planetnum][time]['mass'] = pl[1]
                data[planetnum][time]['a'] = pl[2]
                data[planetnum][time]['e'] = pl[3]
                
      
    sys = dict()
    te = dict()
    
    am = []
    ps = []

    for p in range(1,11):
        ps.append(p)
        am.append(data[p][0]['mass'])                

    lm = ps[am.index(np.max(am))]

    am.remove(np.max(am))
    ps.remove(lm)

    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)

    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)

    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    
    
    for planet in range (1,11):
        sys[planet] = dict() 
        te[planet] = dict()

        te[planet]['a'] = []
        te[planet]['e'] = []
        te[planet]['pericenter'] = []
        te[planet]['log mass1_ratio'] = []
        te[planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[planet])):
            te[planet]['a'].append(float(data[planet][timestep]['a']))
            te[planet]['e'].append(float(data[planet][timestep]['e']))

            massPL = float(data[planet][timestep]['mass'])
            aPL = float(data[planet][timestep]['a'])
            ePL = float(data[planet][timestep]['e'])
            te[planet]['pericenter'].append(aPL*(1-ePL))    
                
            try:    
                if (planet == lm2):
                    m1 = float(data[lm3][0]['mass'])
                    a1 = float(data[lm3][timestep]['a'])

                    m2 = float(data[lm4][0]['mass'])
                    a2 = float(data[lm4][timestep]['a'])

                elif (planet == lm3):
                    m1 = float(data[lm2][0]['mass'])
                    a1 = float(data[lm2][timestep]['a'])

                    m2 = float(data[lm4][0]['mass'])
                    a2 = float(data[lm4][timestep]['a'])

                else:
                    m1 = float(data[lm2][0]['mass'])
                    a1 = float(data[lm2][timestep]['a'])

                    m2 = float(data[lm3][0]['mass'])
                    a2 = float(data[lm3][timestep]['a'])

                te[planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                
                
            except:
                pass
            

    for planet in range(1,11):
        sys[planet]['sd e'] = []
        sys[planet]['max e'] = []
        sys[planet]['cv pericenter'] = []
        sys[planet]['log mass1_ratio'] = []
        sys[planet]['log mass2_ratio'] = []
        
        temp = []
        for i in range(len(data[planet])-10):
            sys[planet]['sd e'].append(np.std(te[planet]['e'][i:i+10]))
            sys[planet]['cv pericenter'].append(np.std(te[planet]['pericenter'][i:i+10])/np.mean(te[planet]['pericenter'][i:i+10]))
            
            temp.append(te[planet]['e'][i])
            sys[planet]['max e'].append(np.max(temp))
            
            try:
                sys[planet]['log mass1_ratio'].append(np.average(te[planet]['log mass1_ratio'][i:i+10]))
                sys[planet]['log mass2_ratio'].append(np.average(te[planet]['log mass2_ratio'][i:i+10]))
            except:
                pass
                
                
    for planet in range(1,11):
        sys[planet]['fate'] = 'remaining'

        with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/info.out', 'r') as file:
            lines = file.readlines()
            for line in lines:
                if "PL" in line:
                    cols = line.split()
                    plnt = int(cols[0].split('L')[-1])
                    plnt2 = cols[4].split('L')[-1]

                    if "ejected" in line:
                        sys[plnt]['fate'] = "ejected"
                    elif "collided with the central body" in line:
                        sys[plnt]['fate'] = "central body collision"
                    elif "was hit by" in line:
                        sys[int(plnt2)]['fate'] = "planet-planet collision"
                    
                    
                 
    fig, axs = plt.subplots(5, 2, figsize=(22,17))#, sharey='row'
    plt.suptitle('System ' + str(rn), fontweight='bold')

    for f in range(len(feats)):
        for p in range(1,11):
            col = 'white'
            zord = 1
            alp = 1
            lw = 1.25

            if (all_dict[rn, p]['outcome'] == all_dict[rn, p]['predict']):
                if (all_dict[rn, p]['outcome'] == 1): # unstable
                    col = 'lightsteelblue'
                else:
                    col = 'darkgoldenrod'  
            else:
                zord = 3
                if (all_dict[rn, p]['outcome'] == 1): # true unstable, predicted stable
                    col = 'mediumseagreen'
                else:
                    col = 'coral'
                    
            if (all_dict[rn, p]['type'] == 'train'):
                alp = 0.7
            
            
            xs2 = []

            for i in range(len(sys[p][feats[f]])):
                xs2.append(i * 13.689)

                
            if (feats[f] == 'log mass2_ratio' or feats[f] == 'log mass1_ratio'):
                lw = 2
                
            axs[f, 0].plot(xs2, sys[p][feats[f]], color = col, zorder = zord, alpha = alp, linewidth = lw)
            axs[f, 1].plot(xs2, sys[p][feats[f]], color = col, zorder = zord, alpha = alp, linewidth = lw)
                
    
    for i in range(len(feats)):
        axs[i, 0].set_xlim(-2, 125)

    for i in range(len(feats)):
        axs[i, 1].set_xlim(-2, 500000)
        
    axs[0, 0].set_ylabel('log mass1 ratio')
    axs[1, 0].set_ylabel('log mass2 ratio')
    axs[2, 0].set_ylabel('max e')
    axs[3, 0].set_ylabel('sd e')
    axs[4, 0].set_ylabel('cv pericenter')


    plt.tight_layout()
    plt.subplots_adjust(top=0.96)
    
    fil.savefig()
    plt.clf()
    pbar.update()
    
fil.close()

In [203]:
pbar = tqdm(total = 100)

data = dict()

for rn in range(1,101):
    data[rn] = dict()
    
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=36561)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                
    pbar.update()

In [205]:
pbar = tqdm(total = 100)

sys = dict()
te = dict()
    
for rn in range(1,101):
    sys[rn] = dict()
    te[rn] = dict()
    
    am = []
    ps = []

    for p in range(1,11):
        ps.append(p)
        am.append(data[rn][p][0]['mass'])                

    lm = ps[am.index(np.max(am))]

    am.remove(np.max(am))
    ps.remove(lm)

    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)

    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)

    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    
    
    for planet in range (1,11):
        sys[rn][planet] = dict() 
        te[rn][planet] = dict()

        te[rn][planet]['a'] = []
        te[rn][planet]['e'] = []
        te[rn][planet]['pericenter'] = []
        te[rn][planet]['log mass1_ratio'] = []
        te[rn][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn][planet])):
            te[rn][planet]['a'].append(float(data[rn][planet][timestep]['a']))
            te[rn][planet]['e'].append(float(data[rn][planet][timestep]['e']))

            massPL = float(data[rn][planet][timestep]['mass'])
            aPL = float(data[rn][planet][timestep]['a'])
            ePL = float(data[rn][planet][timestep]['e'])
            te[rn][planet]['pericenter'].append(aPL*(1-ePL))    
                
            try:    
                if (planet == lm2):
                    m1 = float(data[rn][lm3][0]['mass'])
                    a1 = float(data[rn][lm3][timestep]['a'])

                    m2 = float(data[rn][lm4][0]['mass'])
                    a2 = float(data[rn][lm4][timestep]['a'])

                elif (planet == lm3):
                    m1 = float(data[rn][lm2][0]['mass'])
                    a1 = float(data[rn][lm2][timestep]['a'])

                    m2 = float(data[rn][lm4][0]['mass'])
                    a2 = float(data[rn][lm4][timestep]['a'])

                else:
                    m1 = float(data[rn][lm2][0]['mass'])
                    a1 = float(data[rn][lm2][timestep]['a'])

                    m2 = float(data[rn][lm3][0]['mass'])
                    a2 = float(data[rn][lm3][timestep]['a'])

                te[rn][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                
                
            except:
                pass
            
    pbar.update()

In [224]:
pbar = tqdm(total = 100)

for rn in range(1,101):
    for planet in range(1,11):
        sys[rn][planet]['sd e'] = []
        sys[rn][planet]['max e'] = []
        sys[rn][planet]['cv pericenter'] = []
        sys[rn][planet]['log mass1_ratio'] = []
        sys[rn][planet]['log mass2_ratio'] = []
        
        temp = []
        for i in range(len(data[rn][planet])-10):
            sys[rn][planet]['sd e'].append(np.std(te[rn][planet]['e'][i:i+10]))
            sys[rn][planet]['cv pericenter'].append(np.std(te[rn][planet]['pericenter'][i:i+10])/np.mean(te[rn][planet]['pericenter'][i:i+10]))
            
            temp.append(te[rn][planet]['e'][i])
            sys[rn][planet]['max e'].append(np.max(temp))
            
            try:
                sys[rn][planet]['log mass1_ratio'].append(np.average(te[rn][planet]['log mass1_ratio'][i:i+10]))
                sys[rn][planet]['log mass2_ratio'].append(np.average(te[rn][planet]['log mass2_ratio'][i:i+10]))
            except:
                pass
                
                
    for planet in range(1,11):
        sys[rn][planet]['fate'] = 'remaining'

        with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/info.out', 'r') as file:
            lines = file.readlines()
            for line in lines:
                if "PL" in line:
                    cols = line.split()
                    plnt = int(cols[0].split('L')[-1])
                    plnt2 = cols[4].split('L')[-1]

                    if "ejected" in line:
                        sys[plnt]['fate'] = "ejected"
                    elif "collided with the central body" in line:
                        sys[plnt]['fate'] = "central body collision"
                    elif "was hit by" in line:
                        sys[int(plnt2)]['fate'] = "planet-planet collision"
                        
    pbar.update()

/Users/neha/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/neha/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [231]:
feats = ['log mass1_ratio', 'log mass2_ratio', 'max e', 'sd e', 'cv pericenter']

pbar = tqdm(total = 100)
fil = PdfPages('time stamping revised all feats.pdf')

for rn in range(1,101):
    fig, axs = plt.subplots(5, 2, figsize=(22,17))#, sharey='row'
    plt.suptitle('System ' + str(rn), fontweight='bold')

    for f in range(len(feats)):
        for p in range(1,11):
            col = 'white'
            zord = 1
            alp = 1
            lw = 1.25

            if (all_dict[rn, p]['outcome'] == all_dict[rn, p]['predict']):
                if (all_dict[rn, p]['outcome'] == 1): # unstable
                    col = 'thistle'
                else:
                    col = 'cornflowerblue'  
            else:
                zord = 3
                if (all_dict[rn, p]['outcome'] == 1): # true unstable, predicted stable
                    col = 'palevioletred'
                else:
                    col = 'mediumseagreen'
                    
            if (all_dict[rn, p]['type'] == 'train'):
                alp = 0.7
            
            
            xs2 = []

            for i in range(len(sys[rn][p][feats[f]])):
                xs2.append(i * 13.689)

                
            if (feats[f] == 'log mass2_ratio' or feats[f] == 'log mass1_ratio'):
                lw = 2
                
            axs[f, 0].plot(xs2, sys[rn][p][feats[f]], color = col, zorder = zord, alpha = alp, linewidth = lw)
            axs[f, 1].plot(xs2, sys[rn][p][feats[f]], color = col, zorder = zord, alpha = alp, linewidth = lw)
                
    
    for i in range(len(feats)):
        axs[i, 0].set_xlim(-2, 125)

    for i in range(len(feats)):
        axs[i, 1].set_xlim(-2, 500100)
        
    axs[0, 0].set_ylabel('log mass1 ratio')
    axs[1, 0].set_ylabel('log mass2 ratio')
    axs[2, 0].set_ylabel('max e')
    axs[3, 0].set_ylabel('sd e')
    axs[4, 0].set_ylabel('cv pericenter')


    plt.tight_layout()
    plt.subplots_adjust(top=0.96)
    
    fil.savefig()
    plt.clf()
    pbar.update()
    
fil.close()

<ipython-input-231-26a30d625914>:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(5, 2, figsize=(22,17))#, sharey='row'


<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

...

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

### without mass ratios *

In [221]:
feats = ['max e', 'sd e', 'cv pericenter']


pbar = tqdm(total = 100)
fil = PdfPages('time stamping revised no mass ratios.pdf')

for rn in range(1,101):
    fig, axs = plt.subplots(3, 2, figsize=(22,17))#, sharey='row'
    plt.suptitle('System ' + str(rn), fontweight='bold')

    for f in range(len(feats)):
        for p in range(1,11):
            col = 'white'
            zord = 1
            alp = 1
            lw = 1.25

            if (all_dict[rn, p]['outcome'] == all_dict[rn, p]['predict']):
                if (all_dict[rn, p]['outcome'] == 1): # unstable
                    col = 'lightsteelblue'
                else:
                    col = 'darkgoldenrod'  
                    zord = 3
            else:
                zord = 5
                if (all_dict[rn, p]['outcome'] == 1): # true unstable, predicted stable
                    col = 'mediumseagreen'
                else:
                    col = 'coral'
                    
            if (all_dict[rn, p]['type'] == 'train'):
                alp = 0.7
            
            
            xs2 = []

            for i in range(len(sys[rn][p][feats[f]])):
                xs2.append(i * 13.689)

                
            axs[f, 0].plot(xs2, sys[rn][p][feats[f]], color = col, zorder = zord, alpha = alp, linewidth = lw)
            axs[f, 1].plot(xs2, sys[rn][p][feats[f]], color = col, zorder = zord, alpha = alp, linewidth = lw)
                
    
    for i in range(len(feats)):
        axs[i, 0].set_xlim(-2, 125)

    for i in range(len(feats)):
        axs[i, 1].set_xlim(-2, 500000)
        
    axs[0, 0].set_ylabel('max e')
    axs[1, 0].set_ylabel('sd e')
    axs[2, 0].set_ylabel('cv pericenter')


    plt.tight_layout()
    plt.subplots_adjust(top=0.96)
    
    fil.savefig()
    plt.clf()
    pbar.update()
    
fil.close()

<ipython-input-221-91a68eeb45a0>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(3, 2, figsize=(22,17))#, sharey='row'


<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

...

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>

<Figure size 1584x1224 with 0 Axes>